In [1]:
from nixtlats import TimeGPT
# import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import config as config

In [2]:
timegpt = TimeGPT(
    # defaults to os.environ.get("TIMEGPT_TOKEN")
    token = "enter API Key from NIXTLA"
)

### Test the validate of your token calling the validate_token method:

In [3]:
timegpt.validate_token()

ApiError: status_code: 429, body: {'data': None, 'message': 'Too many requests', 'details': 'You have reached your request limit, email ops@nixtla.io to continue using the API', 'code': 'A20', 'requestID': 'VXN9AWNDVG', 'support': 'If you have questions or need support, please email ops@nixtla.io'}

Now we can start making forecasts! Let’s import an example on the [Time Series Room Temperature Data](https://www.kaggle.com/datasets/vitthalmadane/ts-temp-1) dataset. Dataset is generated with help of an IOT Device data represents room air temperature values with respect time. First, let’s load the dataset and plot it:

In [ ]:
df = pd.read_csv("data/MLTempDataset.csv")
df.head()

,Unnamed: 0,Datetime1,DAYTON_MW,Datetime
0,0,0,20.867,2022-01-04 00:00:00
1,1,1,21.000,2022-01-04 01:00:00
2,2,2,20.867,2022-01-04 02:00:00
3,3,3,20.650,2022-01-04 03:00:00
4,4,4,20.400,2022-01-04 04:00:00


In [ ]:
timegpt.plot(df, time_col='Datetime', target_col='DAYTON_MW')

ImportError: matplotlib is not installed. Please install it and try again.
You can find detailed instructions at https://matplotlib.org/stable/users/installing/index.html

> Important requirements of the data

1. Make sure the target variable column does not have missing or non-numeric values.
2. Do not include gaps/jumps in the datestamps (for the given frequency) between the first and late datestamps. The forecast function will not impute missing dates.
3. The format of the datestamp column should be readable by Pandas [see this link for more details](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html).
    
> Next, forecast the next 12 months using the SDK forecast method. Set the following parameters:
* df: A pandas dataframe containing the time series data.
* h: The number of steps ahead to forecast.
* freq: The frequency of the time series in Pandas format. See [pandas’ available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
* time_col: Column that identifies the datestamp column.
* target_col: The variable that we want to forecast.



In [ ]:
timegpt_fcst_df = timegpt.forecast(df=df, h=12, freq='MS', time_col='datetime', target_col='DAYTON_MW')
timegpt_fcst_df.head()

INFO:nixtlats.timegpt:Validating inputs...


AttributeError: 'Series' object has no attribute 'ds'

In [ ]:

timegpt.plot(df, timegpt_fcst_df, time_col='datetime', target_col='DAYTON_MW')

In [ ]:
timegpt_fcst_df = timegpt.forecast(df=df, h=36, time_col='datetime', target_col='DAYTON_MW', freq='MS')
timegpt_fcst_df.head()

In [ ]:

timegpt.plot(df, timegpt_fcst_df, time_col='datetime', target_col='DAYTON_MW')

Or a shorter one:

In [ ]:
timegpt_fcst_df = timegpt.forecast(df=df, h=6, time_col='datetime', target_col='DAYTON_MW', freq='MS')
timegpt.plot(df, timegpt_fcst_df, time_col='datetime', target_col='DAYTON_MW')


**Warning**

> TimeGPT-1 is currently optimized for short horizon forecasting. While
the forecast mehtod will allow any positive and large horizon, the
accuracy of the forecasts might degrade.

### Using DateTime index to infer frequency

> The freq parameter, which indicates the time unit between consecutive data points, is particularly critical. Fortunately, you can pass a DataFrame with a DateTime index to the forecasting method, ensuring that your time series data is equipped with necessary temporal features. By assigning a suitable freq parameter to the DateTime index of a DataFrame, you inform the model about the consistent interval between observations — be it days (‘D’), months (‘M’), or another suitable frequency.

In [ ]:
df_time_index = df.set_index('datetime')
df_time_index.index = pd.DatetimeIndex(df_time_index.index, freq='MS')
timegpt.forecast(df=df, h=36, time_col='datetime', target_col='DAYTON_MW').head()